In [ ]:
"""
Flask Web App using the CAPTCHA Generator module
"""

import secrets
from datetime import timedelta
from flask import Flask, session, send_file, request, render_template_string, redirect, url_for, flash
from captcha_generator import generate_captcha  # Import from our module

app = Flask(__name__)
app.secret_key = secrets.token_hex(16)
app.permanent_session_lifetime = timedelta(minutes=5)

INDEX_HTML = """
<!doctype html>
<html>
<head><title>CAPTCHA Demo</title></head>
<body>
  <h2>CAPTCHA Demo (Modular Version)</h2>
  {% with messages = get_flashed_messages() %}
    {% if messages %}
      <ul>{% for m in messages %}<li style="color:red">{{ m }}</li>{% endfor %}</ul>
    {% endif %}
  {% endwith %}
  <form method="POST" action="{{ url_for('verify') }}">
    <p>
      <img src="{{ url_for('captcha') }}?t={{ token }}" alt="CAPTCHA" style="border:1px solid #ccc">
      <button type="button" onclick="location.reload();">Refresh</button>
    </p>
    <p>
      <label>Enter the CAPTCHA text:</label><br>
      <input type="text" name="captcha_input" autocomplete="off">
    </p>
    <button type="submit">Submit</button>
  </form>
</body>
</html>
"""

@app.route('/')
def index():
    token = secrets.token_urlsafe(8)
    session['captcha_token'] = token
    return render_template_string(INDEX_HTML, token=token)

@app.route('/captcha')
def captcha():
    text, buf = generate_captcha()
    session['captcha_text'] = text
    return send_file(buf, mimetype='image/png')

@app.route('/verify', methods=['POST'])
def verify():
    user_input = (request.form.get('captcha_input') or '').strip().upper()
    expected = (session.get('captcha_text') or '').strip().upper()
    session.pop('captcha_text', None)
    if not user_input:
        flash('Please enter the CAPTCHA text.')
    elif user_input == expected and expected != '':
        return '<h3 style="color:green">CAPTCHA verified successfully ✅</h3>'
    else:
        flash('Incorrect CAPTCHA, try again.')
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)
